# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [56]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,34.65,90,93,37.11,TF,1720309975
1,1,college,64.8569,-147.8028,60.75,84,100,13.80,US,1720309976
2,2,inari,68.9060,27.0288,55.96,78,99,11.30,FI,1720309977
3,3,hamilton,39.1834,-84.5333,83.03,58,71,8.01,US,1720309979
4,4,lautoka,-17.6167,177.4667,80.46,57,20,13.80,FJ,1720309980


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [58]:
%%capture --no-display

# Set bounds
mapBounds = {
    'xRange': (-180, 180),
    'yRange': (-90, 90)
}
# Configure the map plot
mapPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    tiles = "OSM",
    frame_width = 900,
    frame_height = 900, 
    scale = 0.75,
    color = "City",
    alpha=0.5,
    geo = True,
    xlim=mapBounds['xRange'],
    ylim=mapBounds['yRange']
)

# Display the map
mapPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria (play around with your own criteria to get a sizable amount of locations)
idealWeatherDF = city_data_df.loc[(city_data_df['Max Temp'] >= 69.8) & (city_data_df['Max Temp'] <= 80.6) & 
                                  (city_data_df['Wind Speed'] <= 10) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
idealWeatherDF = idealWeatherDF.dropna()

# Display sample data
idealWeatherDF

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
72,72,chibougamau,49.9168,-74.3659,74.82,35,0,4.61,CA,1720310065
99,99,grande prairie,55.1667,-118.8027,77.18,31,0,4.61,CA,1720310097
115,115,dauphin,51.1494,-100.0502,69.94,68,0,8.05,CA,1720310117
159,159,arraial do cabo,-22.9661,-42.0278,70.92,83,0,8.93,BR,1720310171
165,165,aral,40.5156,81.2636,75.54,41,0,6.22,CN,1720310181
180,180,great falls,47.5002,-111.3008,77.16,40,0,8.05,US,1720310198
193,193,fort st. john,56.2499,-120.8529,76.89,33,0,6.91,CA,1720310214
254,254,labrador city,52.9463,-66.9114,73.31,25,0,9.22,CA,1720310289
256,256,buka,40.8108,69.1986,70.61,32,0,3.38,UZ,1720310291
329,329,hun,29.1268,15.9477,77.76,54,0,9.33,LY,1720310383


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealWeatherDF[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
72,chibougamau,CA,49.9168,-74.3659,35,
99,grande prairie,CA,55.1667,-118.8027,31,
115,dauphin,CA,51.1494,-100.0502,68,
159,arraial do cabo,BR,-22.9661,-42.0278,83,
165,aral,CN,40.5156,81.2636,41,
180,great falls,US,47.5002,-111.3008,40,
193,fort st. john,CA,56.2499,-120.8529,33,
254,labrador city,CA,52.9463,-66.9114,25,
256,buka,UZ,40.8108,69.1986,32,
329,hun,LY,29.1268,15.9477,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': 'accommodation.hotel',
    'limit': 1,
    'apiKey': geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
chibougamau - nearest hotel: No hotel found
grande prairie - nearest hotel: Sandman Hotel Grande Prairie
dauphin - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
aral - nearest hotel: No hotel found
great falls - nearest hotel: La Quinta Inn & Suites
fort st. john - nearest hotel: Fort St. John Motor Inn
labrador city - nearest hotel: Carol Inn
buka - nearest hotel: No hotel found
hun - nearest hotel: فندق الهروج
mogocha - nearest hotel: 7Ka
vilhena - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
parry sound - nearest hotel: Mid Town Hotel
blair - nearest hotel: Super 8 by Wyndham Blair
pulsano - nearest hotel: Sud
argostolion - nearest hotel: Ainos Hotel
brindisi - nearest hotel: Villa Rosmarino
roccella ionica - nearest hotel: Mediterraneo
nushki - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
72,chibougamau,CA,49.9168,-74.3659,35,No hotel found
99,grande prairie,CA,55.1667,-118.8027,31,Sandman Hotel Grande Prairie
115,dauphin,CA,51.1494,-100.0502,68,No hotel found
159,arraial do cabo,BR,-22.9661,-42.0278,83,No hotel found
165,aral,CN,40.5156,81.2636,41,No hotel found
180,great falls,US,47.5002,-111.3008,40,La Quinta Inn & Suites
193,fort st. john,CA,56.2499,-120.8529,33,Fort St. John Motor Inn
254,labrador city,CA,52.9463,-66.9114,25,Carol Inn
256,buka,UZ,40.8108,69.1986,32,No hotel found
329,hun,LY,29.1268,15.9477,54,فندق الهروج


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
%%capture --no-display

# Set bounds
mapBounds = {
    'xRange': (-180, 180),
    'yRange': (-90, 90)
}

# Configure the map plot
hotelPlot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    tiles = "OSM",
    frame_width = 900,
    frame_height = 900, 
    scale = 0.75,
    color = "City",
    alpha=0.5,
    geo = True,
    xlim=mapBounds['xRange'],
    ylim=mapBounds['yRange'],
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotelPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)